
# File processing examples
One of the most common uses of computers is to process file data. 
Much of the data used in data processing starts as text that can be read in files.
File processing programs read data in files, analyze it somehow, and save the data in some useful way.

## Summarizing groups of records
The program below reads the office supply purchase data from the file `file.txt` used above and prints a summary of the purchases with the total cost.

This example uses regular expressions to separate the different items on each record.
These are regular expressions to pull out the date, cost, and name of supplies.
- date `(\d{2})/(\d{2})/(\d{4})`
- cost `\$(\d+\.\d+)` where `\$` and `\.` are escaped characters because `$` and `.` are special characters
- name of supplies `(.*)` where `.*` matches all the text after the cost

`import re` is needed to start using regular expressions. 
The function `m.group(1)` returns the string matching the first part of the
regular expression in parentheses, here `(\d{2}/\d{2}/\d{4})`.

<img src="regex5.jpg" width="400">

In [1]:
import re
p = "(\d{2}/\d{2}/\d{4}) \$(\d+\.\d+) (.*)"
f = open("file.txt", "r")
total = 0.0
for line in f:
    m = re.search(p, line)
    print()
    print(line.strip())
    if m:
        date = m.group(1)
        cost = m.group(2)
        name = m.group(3)
        total += float(cost)
        print("date ", date, " cost $", cost, " name ", name)
    else:
        print("    line is not the right format")
f.close()
print()
print("total cost: $", total)


05/20/2021 $20.11 Office supplies
date  05/20/2021  cost $ 20.11  name  Office supplies

12/01/2021 $123.11 Desk chair
date  12/01/2021  cost $ 123.11  name  Desk chair

06/14/2021 $50.82 Telephone bill
date  06/14/2021  cost $ 50.82  name  Telephone bill

total cost: $ 194.04


## Summarizing groups of records by type
This example uses regular expressions to add the cost of items in the file and accumulating them by type.
The example uses the file `orders.txt`, which is a data source for purchases of cell phone, cable TV, and internet services.
This example Each line has the format:
- date
- customer
- cost
- service purchased

The program takes the date, name, cost, and service purchased and totals the purchases by name and service type.
`orders.txt` contains these lines.

```
01/05/2022 Colin Heath $70.00 Phone-Cell
01/11/2022 Courtney Collins $14.99 Cable-HBO
01/12/2022 John Johnson $70.00 Phone-Cell
01/17/2022 Colin Heath $50.00 Internet-100MB
02/18/2022 John Johnson $80.00 Internet-1GB
02/21/2022 Courtney Collins $6.99 Cable-Hulu
02/23/2022 Colin Heath $6.99 Cable-Hulu
03/07/2022 Courtney Collins $50.00 Internet-100MB
```

These are regular expressions to pull out the date, cost, and name of supplies.
- date `(\d{2})/(\d{2})/(\d{4})`
- first name `(\w+)`
- last name `(\w+)`
- cost `\$(\d+\.\d+)` where `\$` and `\.` are escaped characters because `$` and `.` are special characters
- name of service `(.*)` where `.*` matches all the text after the cost

These are notes about the program.
- The dictionaries *customers* and *services* are used to hold the total cost for the purchases for each person and each service.
- The function `add_to` totals the cost of an item by customer and service. 
- The first time the cost is assigned to the dictionary entry for the item, because the item is not yet in the dictionary. After that it adds the cost to the total for that dictionary item. 
- The results are printed in a readable way using the string function `format`.
- `import re` is needed to start using regular expressions. 

<img src="regex6.jpg" width="400">

In [3]:
import re

def print_totals(item, dict):
    keys = list(dict.keys())
    keys.sort()
    item_header = '{:17s}'.format(item)
    print(item_header, "  costs")
    print("-----------------  --------")
    for key in keys:
        name = key
        total = dict[key]
        print('{:17s}  ${:7.2f}'.format(name, total))

def add_to(customers, name, services, item, cost):
    if name in customers:
        customers[name] += cost
    else:
        customers[name] = cost
    if item in services:
        services[item] += cost
    else:
        services[item] = cost

p = "(\d{2}/\d{2}/\d{4}) (\w+) (\w+) \$(\d+\.\d+) (.*)"
f = open("orders.txt", "r")
customers = {}
services = {}
print('date         name                 service         cost')
print('------------ -------------------- --------------- --------')
for s in f:
    line = s.strip()
    m = re.search(p, line)
    if m:
        date = m.group(1)
        first = m.group(2)
        last = m.group(3)
        full_name = f'{last}, {first}'
        cost = m.group(4)
        cost_float = float(cost)
        service = m.group(5)
        print('{:12s} {:20s} {:15s} ${:7.2f}'.format(date, full_name, service, cost_float))
        add_to(customers, full_name, services, service, cost_float)
    else:
        print("    line is not the right format")
        print(line)
f.close()
print()
print("total cost for customers")
print_totals("customers", customers)
print()
print("total cost for services")
print_totals("services", services)


date         name                 service         cost
------------ -------------------- --------------- --------
01/05/2022   Heath, Colin         Phone-Cell      $  70.00
01/11/2022   Collins, Courtney    Cable-HBO       $  14.99
01/12/2022   Johnson, John        Phone-Cell      $  70.00
01/17/2022   Heath, Colin         Internet-100MB  $  50.00
02/18/2022   Johnson, John        Internet-1GB    $  80.00
02/21/2022   Collins, Courtney    Cable-Hulu      $   6.99
02/23/2022   Heath, Colin         Cable-Hulu      $   6.99
03/07/2022   Collins, Courtney    Internet-100MB  $  50.00

total cost for customers
customers           costs
-----------------  --------
Collins, Courtney  $  71.98
Heath, Colin       $ 126.99
Johnson, John      $ 150.00

total cost for services
services            costs
-----------------  --------
Cable-HBO          $  14.99
Cable-Hulu         $  13.98
Internet-100MB     $ 100.00
Internet-1GB       $  80.00
Phone-Cell         $ 140.00


## *Joining* two data sources
Often program data comes in two or more data sources. 
These may be in files, or in <a href="#glossary_database">*databases*</a>,
where the data are records containing different data items.
Two data sources may have records that describe the same thing.

This example combines data describing movies in two files.
The example uses the file `movies.txt`, which is a data source for movies and when they were released, and the file `actors.txt`, which is a data source for the actors
in those movies.

Each line of `movies.txt` has the format:
- name of the movie
- date released

Each line of `actors.txt` has the format:
- name of an actor
- a movie they starred in

The program combines the data in both files by taking a movie name from `actors.txt` and finding the movie's release date from `movies.txt`, 
and printing one line with these data items.
- actor
- movie
- release date

The program pulls out the actor, movie, and release date from the records, 
but we want to allow spaces in the movie name so we separate the data items
with `,` instead of a space.

`movies.txt` contains these lines.
```
Forrest Gump,1994
Kate & Leopold,1998
```

`actors.txt` contains these lines.
```
Meg Ryan,Kate & Leopold
Tom Hanks,Forrest Gump
```

The actors dictionary is this.

<img src="actors1.jpg" width="700">

The movies dictionary is this.

<img src="movies1.jpg" width="700">

The actors and movies dictionaries are joined by the movie.

<img src="join2.jpg" width="400">

In [ ]:
def print_actors(movies, actors):
    actor_names = list(actors.keys())
    actor_names.sort()
    print("actor         movie              release date")
    print("------------  -----------------  ------------")
    for actor in actor_names:
        movie = actors[actor]
        release_date = movies[movie]
        print('{:12s}  {:17s}  {:4s}'.format(actor, movie, release_date))

def add_actor(actors, line):
    items = line.split(",")
    actor = items[0]
    movie = items[1]
    actors[actor] = movie

def add_movie(movies, line):
    items = line.split(",")
    movie = items[0]
    release_date = items[1]
    movies[movie] = release_date

f = open("actors.txt", "r")
actors = {}
for s in f:
    line = s.strip()
    add_actor(actors, line)
f.close()

g = open("movies.txt", "r")
movies = {}
for s in g:
    line = s.strip()
    add_movie(movies, line)
g.close()

print_actors(movies, actors)